In [2]:
import pandas as pd

In [ ]:
enru = pd.read_csv("./lookup results/yandex_lookup_en_ru_lemmas_only_WHOLE_LIST.csv")

In [ ]:
enru.info()

In [ ]:
enru.head(10)

In [ ]:
enrufr_naive = pd.read_csv("./lookup results/naive_transaltion_en_ru_fr_LEMMAS_WHOLE_LIST.csv")
enrufr_naive.head()

In [ ]:
enrufr_naive['russian_word'] = list(enru['local_word'])

In [ ]:
del (enrufr_naive['Unnamed: 0'])
enrufr_naive.head()

In [ ]:
column = ['word','russian_word', 'local_word','examples','local_examples']
enrufr_naive = enrufr_naive.reindex(columns=column)

In [ ]:
enrufr_naive.head(60)

In [ ]:
enrufr_naive.to_csv("enrufr_naive_transation_yandex_lookup_whole_list.csv")

# match systran en ru vs en fr

In [3]:
systran_en_fr = pd.read_csv("systran_en_fr_pos.csv")
systran_en_fr.head()

,Unnamed: 0,word,local_word,pos,examples,local_examples,examples_and_definitions
0,0,word,mot,noun,key word|last word|signal word|first words|,mot clé | dernier mot | mot indicateur | premi...,key word|last word|signal word|first words|ter...
1,1,word,parole,noun,fine words|word of god|kind word|word of the l...,belle parole | parole de dieu | aimables parol...,fine words|word of god|kind word|word of the l...
2,2,word,terme,noun,following words|search word|operative word|eng...,termes suivants | terme de recherche | terme p...,following words|search word|operative word|eng...
3,3,word,propos,noun,written word|wise words|clear words|warm words|,propos écrits ici | propos sages | propos clai...,written word|wise words|clear words|warm words...
4,4,word,mention,noun,words of limitation|,mention de limitation |,words of limitation|particular | reference | m...


In [4]:
systran_en_ru = pd.read_csv("systran_en_ru_pos.csv")
systran_en_ru.head(20)

,Unnamed: 0,word,local_word,pos,examples,local_examples,examples_and_definitions
0,0,word,слово,noun,key word|english word|first word|good word|,ключевое слово | английское слово | первое сло...,key word|english word|first word|good word|spe...
1,1,and,и,conj,empty_cell,empty_cell,empty_cell
2,2,and,а,conj,empty_cell,empty_cell,but | while |
3,3,and,и,conj,He thinks of you day and night|,Он думает о вас день и ночь |,He thinks of you day and night|
4,4,and,но,conj,It's an embarrassing joke and not an insult|,"Это неуместная шутка, а не оскорбление |",It's an embarrassing joke and not an insult|
5,5,and,а,conj,This is a squid and not an octopus|,"Это кальмар, а не осьминог |",This is a squid and not an octopus|
6,6,that,так,conj,empty_cell,empty_cell,as | so | as well as | how | so that | both |
7,7,that,то,conj,empty_cell,empty_cell,empty_cell
8,8,that,который / кто / тот который,pron,I know the guy that sold you the disk|,"Я знаю парня, который продал тебе диск |",I know the guy that sold you the disk|
9,9,that,тот,pron,On that evening I met Natalie|Do you see those...,В тот вечер я встретил Натали | Видишь те две ...,On that evening I met Natalie|Do you see those...


In [5]:
import gensim.downloader as api
from sklearn.metrics.pairwise import cosine_distances
from  sklearn.metrics.pairwise import cosine_similarity

from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()
from nltk.corpus import stopwords
stopWords = set(stopwords.words('english'))
import math
import numpy as np

from tqdm import tqdm

In [55]:
!pip3 install word2vec


     |████████████████████████████████| 61kB 985kB/s eta 0:00:011
  Stored in directory: /Users/nigula/Library/Caches/pip/wheels/ef/9f/06/aec42532c9c37e05f936d4d586b15cfdfc9f2ffb62bd7fed1c
Successfully built word2vec


In [56]:
import word2vec

In [58]:
model = word2vec.load('/Users/nigula/Downloads/fr/fr.bin')

ValueError: invalid literal for int() with base 10: b'\x80\x02cgensim.models.word2vec'

In [6]:
model = api.load('glove-twitter-100')

In [9]:
l1 = "hello my friends"
l2 = "hello my girlfriend"

def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None
    
def get_nonstopwords_vectors_sum(lemm_list):
    orig_vector = np.zeros(shape = 100)
    for lemma in lemm_list:
        if lemma not in stopWords:
            #print (lemma)
            try:
                orig_vector += model[lemma]
            except:
                pass
    return orig_vector


def get_lemmatized_line(line):
    pos_tagged_ngramm = pos_tag(line.split())
    lemmatized_line_list = []
    for word_el in pos_tagged_ngramm:
        pos = get_wordnet_pos(word_el[1])
        if pos:
            lemma = lemmatizer.lemmatize(word_el[0], pos =pos)
        else:
            lemma = word_el[0]
        lemmatized_line_list.append(lemma)
    return lemmatized_line_list

get_lemmatized_line(l1)

def find_lines_similarity(line_1, line_2):
    lemmatized_l1_list = get_lemmatized_line(line_1)
    lemmatized_l2_list = get_lemmatized_line(line_2)
    #print(lemmatized_l1_list,lemmatized_l2_list )
    l1_vect = get_nonstopwords_vectors_sum(lemmatized_l1_list)
    l2_vect = get_nonstopwords_vectors_sum(lemmatized_l2_list)
    cos_sim = cosine_similarity(l1_vect.reshape(1, -1), l2_vect.reshape(1, -1))[0][0]
    #print(cos_sim)
    return cos_sim

find_lines_similarity(l1, l2)    
    

0.9236770700727499

In [7]:
cosine_similarity(model['mother'].reshape(1, -1), model['woman'].reshape(1, -1))[0][0]

0.7815035

In [18]:
def update_enf_word(dct_word, senseword, pos, exmpl):
    #print(senseword)
    if senseword in dct_word:
        if dct_word[senseword]['pos'] == pos:
            if exmpl == "empty_cell":
                #print("EMPTTY")
                pass
            else:
                dct_word[senseword]['examples'] += exmpl
        if dct_word[senseword]['pos'] != pos:
            if exmpl == "empty_cell":
            #print("EMPTTY")
                pass
            else:
                dct_word[senseword + "_" + pos] = {}
                dct_word[senseword + "_" + pos]['examples'] = exmpl            
    else:
        dct_word[senseword] = {'examples':'', 'pos':''}
        if exmpl == "empty_cell":
            #print("EMPTTY")
            dct_word[senseword]['examples'] = ''
        else:
            dct_word[senseword]['pos'] = pos
            dct_word[senseword]['examples'] = exmpl

def csv_to_structure_dict(df):
    new_dict = {}
    for df_ind in range(len(df)):
        df_element = list(df.iloc[df_ind])
        eng_word = df_element[1]
        sense_word = df_element[2]
        pos = df_element[3]
        examples_context = df_element[6]
        #print(df_element)
        if eng_word not in new_dict:
            new_dict[eng_word] = {}
            update_enf_word(new_dict[eng_word],sense_word, pos, examples_context)
        else:
            update_enf_word(new_dict[eng_word],sense_word, pos, examples_context)
    return new_dict
dct = csv_to_structure_dict(systran_en_ru)

In [19]:
dct['dinner']

{'ужин': {'examples': 'romantic dinner|gala dinner|family dinner|themed dinner|supper | meal | ',
  'pos': 'noun'},
 'обед': {'examples': 'business dinner|excellent dinner|lunch | meal | afternoon | lunchtime | ',
  'pos': 'noun'}}

In [20]:
dct_en_fr = csv_to_structure_dict(systran_en_fr)

In [21]:
dct_en_fr['dinner'] 

{'dîner': {'examples': 'gala dinner|romantic dinner|mess dinner|business dinners|diner | ',
  'pos': 'noun'},
 'soir': {'examples': 'evening | night | ', 'pos': 'noun'},
 'souper': {'examples': 'formal dinner|group dinner|supper | diner | ',
  'pos': 'noun'},
 'repas': {'examples': 'theme dinners|medieval dinners|nice dinner|family dinner|meal | food | lunch | ',
  'pos': 'noun'}}

In [22]:
c1 = 'gala dinner|romantic dinner|mess dinner|business dinners|'
c2 = 'romantic dinner|gala dinner|family dinner|themed dinner|'
def get_context_similarity(ctx1, ctx2):
    simialrity_list = []
    #print("ctxs","\n", ctx1,"\n",ctx2 )
    #if "empty_cell" not in ctx1 and "empty_cell" not in ctx2:
    if len(ctx1) != 0 and len(ctx2) != 0:
        for ctx1_el in ctx1.split("|"):
            if ctx1_el.strip() != '':
                #print(ctx1_el)
                for ctx1_e2 in ctx2.split("|"):
                    if ctx1_e2.strip() != '': 
                        #print (ctx1_e2.strip())
                        sim = find_lines_similarity(ctx1_el, ctx1_e2) 
                        #print('SIM',ctx1_el, ctx1_e2,sim)
                        simialrity_list.append(sim)
        #print(simialrity_list)
        return np.mean(simialrity_list)
        #return np.median(simialrity_list)
    else:
        return "no_ctx"
    """
    elif "empty_cell" in ctx1 and "empty_cell" in ctx2:
        return "no_ctx_both"
    elif "empty_cell" in ctx1:
        return "no_ctx_l1"
    elif "empty_cell" in ctx2:
        return "no_ctx_l2"   
    """
get_context_similarity(c1, c2)        

0.8326094228960484

In [ ]:
"""
def average_similarity(ex_sim, ctx_sim,simil_dict, local_word, compared_word):
    print(local_word, compared_word)
    simil_dict[local_word][compared_word] = 'no_data_from_compared_word'
    if ex_sim == "no_ctx" and ctx_sim == "no_ctx":
        print("here==")
        simil_dict[local_word][compared_word] = "no_context_for_merge"
    elif ex_sim != "no_ctx" and ctx_sim == "no_ctx":
        simil_dict[local_word][compared_word] = ex_sim
        print("here!=",ex_sim)
    elif ex_sim == "no_ctx" and ctx_sim != "no_ctx":
        simil_dict[local_word][compared_word] = ctx_sim
        print("here=!",ctx_sim)
    else:
        simil_dict[local_word][compared_word] = (ex_sim + ctx_sim)/2
        print("here!!",ex_sim, ctx_sim, (ex_sim + ctx_sim)/2)
"""

In [90]:
def get_context_and_examples_sim(senseword_dict_l1, senseword_dict_l2, simil_dict_1,simil_dict_2, 
                                 loc_w_1, loc_w_2, print_output = True):
    if print_output: print(loc_w_1, senseword_dict_l1,"\n", loc_w_2, senseword_dict_l2)
    if senseword_dict_l1['examples'] and senseword_dict_l2['examples']:
        similarity = get_context_similarity(senseword_dict_l1['examples'], senseword_dict_l2['examples'])
    else:
        similarity = 0
    if print_output: print(loc_w_1, loc_w_2, "examples_sim", similarity,"\n")
        
    if similarity == "no_ctx":
        simil_dict_1[loc_w_1][loc_w_2] = 0
        simil_dict_2[loc_w_2][loc_w_1] = 0
    else:
        simil_dict_1[loc_w_1][loc_w_2] = similarity
        simil_dict_2[loc_w_2][loc_w_1] = similarity

        
def merge_two_lang_structures(original_eng_word, lang_dict_1, lang_dict_2,eng_word_df, l1_list_for_df,context_l1,  l2_list_for_df, context_l2):
    sim_dict_l1 = {}
    sim_dict_l2 = {}
    for local_1_senseword in lang_dict_1.keys():
        local_1_senseword_pos = lang_dict_1[local_1_senseword]['pos']
        
        for local_2_senseword in lang_dict_2.keys():
            local_2_senseword_pos = lang_dict_2[local_2_senseword]['pos']
            if local_2_senseword_pos == local_1_senseword_pos:
                if local_1_senseword not in sim_dict_l1:
                    sim_dict_l1[local_1_senseword] = {}
                if local_2_senseword not in sim_dict_l2:
                    sim_dict_l2[local_2_senseword] = {}
                get_context_and_examples_sim(lang_dict_1[local_1_senseword], lang_dict_2[local_2_senseword],
                                                  sim_dict_l1,sim_dict_l2, local_1_senseword, local_2_senseword)
            
            #break
        #break
    print("INNER_MERGER_DICT")
    print(sim_dict_l1)
    print()
    print(sim_dict_l2)
    print("=======\n")
    
    #MATCHING STRUCTURES FOR ONE ENGLISH WORD
    l1_to_l2_connect = {}
    l2_to_l1_connect = {}
    create_matching_structure(sim_dict_l1, sim_dict_l2,l1_to_l2_connect, l2_to_l1_connect)  
    merge_entities_lists(l1_to_l2_connect, l2_to_l1_connect, eng_word_df, l1_list_for_df, context_l1, l2_list_for_df, context_l2, original_eng_word)

def merge_two_dct(big_dict_l1, big_dict_l2, print_output = False):
    count = 0
    global_merged_dict= {}
    eng_word_for_df = []
    l1_list_for_df = [] 
    context_from_l1_list = []
    l2_list_for_df = []
    context_from_l2_list = []
    for eng_word in tqdm(list(big_dict_l1.keys())[15:]):
        print("pricessinf word", eng_word)
        #merge_two_lang_structures(eng_word, big_dict_l1[eng_word], big_dict_l2[eng_word],
                                  #eng_word_for_df, l1_list_for_df, 
                                  #context_from_l1_list, l2_list_for_df, context_from_l2_list)
        try:
            merge_two_lang_structures(eng_word, big_dict_l1[eng_word], big_dict_l2[eng_word],
                                      eng_word_for_df, l1_list_for_df, context_from_l1_list, 
                                      l2_list_for_df, context_from_l2_list)
        except:
            print(eng_word, 'MERGE_PROBLEM')
        count += 1
        #if count > 5: break
    print(len(eng_word_for_df), len(l1_list_for_df), len(l2_list_for_df))
    data = pd.DataFrame(list(zip(eng_word_for_df, l1_list_for_df,context_from_l1_list, l2_list_for_df, context_from_l2_list)),columns =['eng_word', 'l1','l1_context', 'l2','l2_context'])
    return  data   
#df_all_words = merge_two_dct(dct,dct_en_fr)  


In [38]:
dct['point']

{'точка': {'examples': 'starting point|access point|percentage point|ideal starting point|dot | spot | pixel | location | outlet | ',
  'pos': 'noun'},
 'пункт': {'examples': 'basis point|turning point|vaccination points|passport control points|item | post | paragraph | clause | pip | outlet | ',
  'pos': 'noun'},
 'момент': {'examples': 'tipping point|key point|important point|certain point|moment | time | thing | date | ',
  'pos': 'noun'},
 'очки': {'examples': 'ACP Point|bonus point|Reward Point|political point|glass | run | sunglasses | score | spectacle | goggle | eyeglass | Lounge Point | ',
  'pos': 'noun'},
 'балл': {'examples': 'penalty point|magnitude | score | vote | ball | grade | ',
  'pos': 'noun'},
 'смысл': {'examples': 'sense | meaning | way | implication | ',
  'pos': 'noun'},
 'дело': {'examples': 'case | affair | business | thing | matter | cause | work | ministry | ',
  'pos': 'noun'},
 'очко': {'examples': 'first point|run | pip | ', 'pos': 'noun'},
 'указывать':

In [53]:
dct_en_fr['dinner'] 

{'dîner': {'examples': 'gala dinner|romantic dinner|mess dinner|business dinners|diner | '},
 'soir': {'examples': 'evening | night | '},
 'souper': {'examples': 'formal dinner|group dinner|supper | diner | '},
 'repas': {'examples': 'theme dinners|medieval dinners|nice dinner|family dinner|meal | food | lunch | '}}

In [83]:
df_all_words.head(40)

,eng_word,l1,l1_context,l2,l2_context
0,treat,gâterie,wholesome treat|sweet treats|,угощение,morning treat|feast |
1,treat,gâterie,wholesome treat|sweet treats|,удовольствие,pleasure | fun | satisfaction | enjoyment | am...
2,treat,soigner,to treat people|to treat angina|cure | heal | ...,лечить,to treat people|heal | cure | heal | dose |
3,treat,soigner,to treat people|to treat angina|cure | heal | ...,вылечить,cure | heal |
4,treat,considérer,treated on a case-by-case basis|treated confid...,рассматривать,consider | see | view | regard | examine | rev...
5,treat,considérer,treated on a case-by-case basis|treated confid...,относиться,include | apply | belong | relate | concern | ...
6,treat,considérer,treated on a case-by-case basis|treated confid...,обращаться,turn | contact | apply | appeal | address | de...
7,treat,considérer,treated on a case-by-case basis|treated confid...,рассматриваться,consider | view | address | examine | regard |...
8,treat,traiter,to treat cancer|to treat disease|to treat diab...,обрабатывать,to treat even vast skin areas|process | handle...
9,treat,friandise,real treat|natural pet treats|sweet | candy | ...,угощение,morning treat|feast |


In [31]:
df_all_words.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3076 entries, 0 to 3075
Data columns (total 5 columns):
eng_word      3076 non-null object
l1            3076 non-null object
l1_context    2310 non-null object
l2            3076 non-null object
l2_context    2410 non-null object
dtypes: object(5)
memory usage: 120.2+ KB


In [40]:
df_all_words.to_csv("systran_merge_mean_04_similarity_vs_pos.csv")

In [23]:
def merge_dicts(l1_l2_dict,l2_to_l1, l1_to_l2):
    for local_word_1 in l1_l2_dict.keys():
        sorted_sim = sorted(l1_l2_dict[local_word_1].items(), key=lambda kv: kv[1], reverse = True)
        if sorted_sim[0][1] < 0.4:
            print("no suitable connections for ", local_word_1)
            l1_to_l2[local_word_1] = ["no_equality"]
        else:
            if local_word_1 in l1_to_l2:
                if sorted_sim[0][0] not in l1_to_l2[local_word_1]:
                    l1_to_l2[local_word_1].append(sorted_sim[0][0])
            else:
                l1_to_l2[local_word_1] = []
                l1_to_l2[local_word_1].append(sorted_sim[0][0])
            print(local_word_1, "sorted_sim[0][1]",sorted_sim[0][1])
            
            if sorted_sim[0][0] in l2_to_l1:  
                if local_word_1 not in l2_to_l1[sorted_sim[0][0]]:
                    l2_to_l1[sorted_sim[0][0]].append(local_word_1)
            else:
                l2_to_l1[sorted_sim[0][0]] = []
                l2_to_l1[sorted_sim[0][0]].append(local_word_1)
                
ru_fr_dict = {'обойти': {'se promener': 0.16193252812028638, 'se balader': 0.15641367945030674, "s'éloigner": 0.162819447439424, 'promener': 0.16490256085461272}, 'вращаться': {'se promener': 0.4123024692238632, 'se balader': 0.36482488114980133, "s'éloigner": 0.19768349945429559, 'promener': 0.22889580396221318}, 'ездить по миру': {'se promener': 0, 'se balader': 0, "s'éloigner": 0, 'promener': 0}, 'часто путешествовать': {'se promener': 0, 'se balader': 0, "s'éloigner": 0, 'promener': 0}, 'хватать': {'se promener': 0.36273753561810446, 'se balader': 0.2975160600112141, "s'éloigner": 0.3047695342194968, 'promener': 0.29944649457673916}, 'пролетать': {'se promener': 0.6350658803284455, 'se balader': 0.3733952107024293, "s'éloigner": 0.4256126771370779, 'promener': 0.6414917193832451}, 'ходить вокруг': {'se promener': 0, 'se balader': 0, "s'éloigner": 0, 'promener': 0}}
fr_ru_dict = {'se promener': {'обойти': 0.16193252812028638, 'вращаться': 0.4123024692238632, 'ездить по миру': 0, 'часто путешествовать': 0, 'хватать': 0.36273753561810446, 'пролетать': 0.6350658803284455, 'ходить вокруг': 0}, 'se balader': {'обойти': 0.15641367945030674, 'вращаться': 0.36482488114980133, 'ездить по миру': 0, 'часто путешествовать': 0, 'хватать': 0.2975160600112141, 'пролетать': 0.3733952107024293, 'ходить вокруг': 0}, "s'éloigner": {'обойти': 0.162819447439424, 'вращаться': 0.19768349945429559, 'ездить по миру': 0, 'часто путешествовать': 0, 'хватать': 0.3047695342194968, 'пролетать': 0.4256126771370779, 'ходить вокруг': 0}, 'promener': {'обойти': 0.16490256085461272, 'вращаться': 0.22889580396221318, 'ездить по миру': 0, 'часто путешествовать': 0, 'хватать': 0.29944649457673916, 'пролетать': 0.6414917193832451, 'ходить вокруг': 0}}
def create_matching_structure(l1_l2_dict, l2_l1_dict, l1_to_l2, l2_to_l1):
    print("FIRST LANG TO SECOND")
    merge_dicts(l1_l2_dict, l1_to_l2, l2_to_l1)
    """
    for local_word_1 in l1_l2_dict.keys():
        sorted_sim = sorted(l1_l2_dict[local_word_1].items(), key=lambda kv: kv[1], reverse = True)
        print(local_word_1,)
        if sorted_sim[0][1] == "no_context_for_merge":
            l1_to_l2[local_word_1] = "no_equality"
        elif sorted_sim[0][1] <= 0.6:
            l1_to_l2[local_word_1] = "no_equality"
        else:
            l1_to_l2[local_word_1] = sorted_sim[0][0]
            #print(local_word_1, "sorted_sim[0][1]",sorted_sim[0][1])
            l2_to_l1[sorted_sim[0][0]] = local_word_1
    """
    print('\n',l1_to_l2)
    print()
    print(l2_to_l1)
    print("\nSECOND LANG TO FIRST")
    merge_dicts(l2_l1_dict, l2_to_l1, l1_to_l2)
    print('\n',l1_to_l2)
    print()
    print(l2_to_l1)
    print()
    

l1_to_l2_connect = {}
l2_to_l1_connect = {}

create_matching_structure(ru_fr_dict, fr_ru_dict,l1_to_l2_connect, l2_to_l1_connect)    
    

FIRST LANG TO SECOND
no suitable connections for  обойти
вращаться sorted_sim[0][1] 0.4123024692238632
no suitable connections for  ездить по миру
no suitable connections for  часто путешествовать
no suitable connections for  хватать
пролетать sorted_sim[0][1] 0.6414917193832451
no suitable connections for  ходить вокруг

 {'se promener': ['вращаться'], 'promener': ['пролетать']}

{'обойти': ['no_equality'], 'вращаться': ['se promener'], 'ездить по миру': ['no_equality'], 'часто путешествовать': ['no_equality'], 'хватать': ['no_equality'], 'пролетать': ['promener'], 'ходить вокруг': ['no_equality']}

SECOND LANG TO FIRST
se promener sorted_sim[0][1] 0.6350658803284455
no suitable connections for  se balader
s'éloigner sorted_sim[0][1] 0.4256126771370779
promener sorted_sim[0][1] 0.6414917193832451

 {'se promener': ['вращаться', 'пролетать'], 'promener': ['пролетать'], 'se balader': ['no_equality'], "s'éloigner": ['пролетать']}

{'обойти': ['no_equality'], 'вращаться': ['se promener'],

In [ ]:
l1_to_l2_connect

In [ ]:
l2_to_l1_connect

In [87]:
def get_context_from_global_variable_dict(eng_word,local_word):
    #print(dct[eng_word])
    #print(dct_en_fr[eng_word])
    l1_sensewords = list(dct_en_ru_azure[eng_word].keys())
    l2_sensewords = list(dct_en_fr_azure[eng_word].keys())
    if local_word == "no_equality":
        pass
    elif local_word in l1_sensewords:
        print("all_clear", local_word)
        return dct_en_ru_azure[eng_word][local_word]['examples']
    elif local_word in l2_sensewords:
        print("all_clear", local_word)
        return dct_en_fr_azure[eng_word][local_word]['examples']
    else:
        print("something_went_ne_tak_no_word_in_dict", local_word)
    #print(l1_sensewords)
    #print(l2_sensewords)
get_context_from_global_variable_dict("treat", "угощение")    
    

all_clear угощение


"treat | refreshments | I just want my treat. | A simple but delightfully sweet treat. | Take the shortcuts and play trick or treat with neighbors! | What about our treat! | Open the door and see your treat. | And she always gets a treat. | I got a special treat for you. | ... guest and they gave me a treat. | ... a restaurant hit, and treat for the family table. | My treat, my treat. | My treat, my treat. | A real chocolate treat for demanding chocolate fans! | This is kind of a treat from the restaurant. | All the kinsfolk and friends gather for the treat. | This morning treat is served in the villas' ancient cellars ... | "

In [33]:
def handle_pairs(l1_to_l2, handled_pairs, eng_word_list, l1_list,l1_cont, l2_list,l2_cont, english_word):
    for l1_word in l1_to_l2.keys():
        print(l1_word)
        for l2_word in l1_to_l2[l1_word]:
            pair = l1_word + "_" + l2_word
            if pair not in handled_pairs:
                eng_word_list.append(english_word)
                l1_list.append(l1_word)
                l1_context = get_context_from_global_variable_dict(english_word, l1_word)    
                l1_cont.append(l1_context)
                l2_list.append(l2_word)
                l2_context = get_context_from_global_variable_dict(english_word, l2_word) 
                l2_cont.append(l2_context)
                handled_pairs.append(pair)

In [ ]:
merge_entities_lists(l1_to_l2_connect, l2_to_l1_connect, eng_word_df, l1_list_for_df, l2_list_for_df)

In [32]:
def merge_entities_lists(l1_to_l2, l2_to_l1, eng_word_list, l1_list, l1_context, l2_list, l2_context, english_word):
    handled_pairs = []
    handle_pairs(l1_to_l2,handled_pairs,eng_word_list,l1_list,l1_context, l2_list, l2_context, english_word)
    handle_pairs(l2_to_l1,handled_pairs,eng_word_list, l2_list, l2_context, l1_list,l1_context,  english_word)                  
    

In [ ]:
def merge_entities(l1_to_l2, l2_to_l1):
    handled_pairs = []
    l1_line = []
    l2_line = []
    handle_pairs(l1_to_l2,handled_pairs,l1_line,l2_line)
    handle_pairs(l2_to_l1,handled_pairs,l2_line, l1_line)                  
    
    data = pd.DataFrame(list(zip(l1_line,l2_line)),columns =['l1', 'l2'])
    return  data

        
        
df_fin = merge_entities(l1_to_l2_connect, l2_to_l1_connect)
df_fin.head()       

In [ ]:
x = {1: 2, 3: 4, 4: 3, 2: 1, 0: 0}
sorted(x.items(), key=lambda kv: kv[1])

# evaluate empty lines

In [46]:
df_mean_4 = pd.read_csv("./reverse_connected_dicts/systran_merge_mean_04_similarity.csv")

In [51]:
df_mean_4.head()

,Unnamed: 0,eng_word,l1,l1_context,l2,l2_context
0,0,treat,gâterie,wholesome treat|sweet treats|,угощение,morning treat|feast |
1,1,treat,gâterie,wholesome treat|sweet treats|,удовольствие,pleasure | fun | satisfaction | enjoyment | am...
2,2,treat,soigner,to treat people|to treat angina|cure | heal | ...,лечить,to treat people|heal | cure | heal | dose |
3,3,treat,soigner,to treat people|to treat angina|cure | heal | ...,вылечить,cure | heal |
4,4,treat,considérer,treated on a case-by-case basis|treated confid...,рассматривать,consider | see | view | regard | examine | rev...


In [57]:
len(df_mean_4[df_mean_4['l1']=="no_equality"]), len(df_mean_4[df_mean_4['l2']=="no_equality"]), len(df_mean_4)

(483, 460, 3465)

In [54]:
def calc_empty_entities(df):
    return (len(df[df['l1']=="no_equality"]) + len(df[df['l2']=="no_equality"]))/len(df)
print(len(df_mean_4))
calc_empty_entities(df_mean_4)



3465


0.27215007215007214

In [53]:
df_mean_4_pos = pd.read_csv("./reverse_connected_dicts/systran_merge_mean_04_similarity_vs_pos.csv")
print(len(df_mean_4_pos))
calc_empty_entities(df_mean_4_pos)

2957


0.24721001014541766

In [48]:
df_med_3 = pd.read_csv("./reverse_connected_dicts/systran_merge_median_03_similarity.csv")
print(len(df_med_3))
calc_empty_entities(df_med_3)

3723


0.17593338705345152

In [50]:
df_med_4 = pd.read_csv("./reverse_connected_dicts/systran_merge_median_04_similarity.csv")
print(len(df_med_4))
calc_empty_entities(df_med_4)

3468


0.2779700115340254

In [52]:
df_med_5 = pd.read_csv("./reverse_connected_dicts/systran_merge_median_05_similarity.csv")
print(len(df_med_5))
calc_empty_entities(df_med_5)


3076


0.46553966189856955

# PROCESS AZURE RESULTS

In [61]:
azure_en_ru = pd.read_csv("lookup_azure_en_ru.csv")
azure_en_ru.head()

,Unnamed: 0,word,local_word,pos,examples,local_examples,examples_and_definitions
0,0,word,слово,NOUN,NOUN,NOUN,word | floor | words | But i need your word. |...
1,1,word,microsoft word,NOUN,NOUN,NOUN,word | Per snapshot image size and placement c...
2,2,word,текстовый,NOUN,NOUN,NOUN,text | word | Word processing applications all...
3,3,word,поговорить,NOUN,NOUN,NOUN,talk | speak | word | chat | I need to have a ...
4,4,the,в,DET,DET,DET,in | the | The restaurant serves dinner as an ...


In [62]:
dct_en_ru_azure = csv_to_structure_dict(azure_en_ru)

In [63]:
azure_en_fr = pd.read_csv("lookup_azure_en_fr.csv")
azure_en_fr.head()

,Unnamed: 0,word,local_word,pos,examples,local_examples,examples_and_definitions
0,0,word,conseil,NOUN,NOUN,NOUN,national security council | council | board | ...
1,1,word,nouvelles,OTHER,OTHER,OTHER,news | word | stop press | breaking news | I w...
2,2,word,mot,NOUN,NOUN,NOUN,line | note | code word | word | password | ro...
3,3,word,paroles,OTHER,OTHER,OTHER,word | talk | lyrics | lyric | pep talk | utte...
4,4,word,parole,OTHER,OTHER,OTHER,word | faith | speech | gospel truth | spoken ...


In [64]:
dct_en_fr_azure = csv_to_structure_dict(azure_en_fr)

In [91]:
df_all_words_azure = merge_two_dct(dct_en_ru_azure,dct_en_fr_azure)  






  0%|          | 0/182 [00:00<?, ?it/s]

pricessinf word necessary
необходимые {'examples': 'necessary | required | needed | essential | appropriate | desired | Continue specifying points to draft the run as necessary. | Make the necessary changes to the display of the part. | Follow the standard installation steps as necessary. | All necessary attribute definitions are automatically placed. | The customer imports the package and installs the necessary certificates. | Check any related items, and perform the necessary procedures. | Includes all necessary mounting for quick installation. | Are you entering all the necessary data? | Create the tasks necessary to implement the change request. | Click edit and make any necessary changes. | Change the sheet properties as necessary. | Choose another file containing all of the necessary data. | Over time, gathered the necessary human potential and knowledge. | Necessary tools of the trade. | Use this time to make necessary changes! | ', 'pos': 'ADJ'} 
 nécessaire {'examples': 'neces

обязательно inévitable examples_sim 0.34508522338489345 

нужно {'examples': 'need | should | want | needs | must | needed | gotta | necessary | Therefore it is necessary to analyse our mistakes. | It is to me necessary to think about it. | If necessary, rename the constructs. | If necessary, we can also assemble it. | It is not necessary to activate them. | I knew, it is necessary to look after you. | Why is it necessary to have my clock movement cleaned? | Create the source dataset if necessary. | They are all used when necessary. | It is not necessary to create query. | I will do whatever is necessary. | There are no additional steps necessary. | Is it necessary to meet colleagues coming from other cities? | And very necessary was this. | This is necessary to prevent damage of teeth. | ', 'pos': 'ADJ'} 
 nécessaire {'examples': 'necessary | writing case | The Army needs men who are willing to fight, when necessary. | It is not necessary to explain everything. | Is it necessary for e

требуется inévitable examples_sim 0.35745347409440903 

надо {'examples': "should | necessary | Nothing is necessary to me. | So what you did was necessary. | Also it is necessary to understand that it means. | You are not told in advance where necessary. | After such bath is necessary to have rest. | I you later way to transfer necessary! | It is necessary to say three times. | The doctor said it is necessary. | Is it necessary to buy workbooks? | Only it is not necessary to swear! | He would have written everything that was necessary. | It is necessary to hold an inquiry on the spot. | It is necessary anymore to know about him. | Although he can be very strict when it's necessary. | I said it's not necessary. | ", 'pos': 'ADJ'} 
 nécessaire {'examples': 'necessary | writing case | The Army needs men who are willing to fight, when necessary. | It is not necessary to explain everything. | Is it necessary for everyone to be there? | We gathered the necessary supplies. | It is necessary 





  1%|          | 1/182 [00:02<08:46,  2.91s/it]

...

до devant examples_sim 0.31244164177633166 

INNER_MERGER_DICT
{'перед': {'devant': 0.3952170511144126}, 'прежде чем': {'devant': 0.42543128594334373}, 'до': {'devant': 0.31244164177633166}}

{'devant': {'перед': 0.3952170511144126, 'прежде чем': 0.42543128594334373, 'до': 0.31244164177633166}}

FIRST LANG TO SECOND
no suitable connections for  перед
прежде чем sorted_sim[0][1] 0.42543128594334373
no suitable connections for  до

 {'devant': ['прежде чем']}

{'перед': ['no_equality'], 'прежде чем': ['devant'], 'до': ['no_equality']}

SECOND LANG TO FIRST
devant sorted_sim[0][1] 0.42543128594334373

 {'devant': ['прежде чем']}

{'перед': ['no_equality'], 'прежде чем': ['devant'], 'до': ['no_equality']}

devant
all_clear devant
all_clear прежде чем
перед
all_clear перед
прежде чем
all_clear прежде чем
all_clear devant
до
all_clear до
pricessinf word begin
INNER_MERGER_DICT
{}

{}

FIRST LANG TO SECOND

 {}

{}

SECOND LANG TO FIRST

 {}

{}

pricessinf word behind
позади {'examples': 'be





 99%|█████████▉| 180/182 [10:54<00:05,  2.77s/it]

за en retard examples_sim 0.5470991099218863 

за {'examples': "for | per | behind | I will not stop until those lips are behind bars! | Just stay behind the yellow tape. | The senior residents stand behind you. | Outside the door, or behind the bar? | Can you just put your hands behind your back? | She did nail him behind your back. | No need to worry about the science behind it. | I prefer staying behind the scenes. | If they remain behind the sun. | You got a glimpse behind the curtain. | And maybe behind that tree over there. | So you're the man behind all this? | Put your hands behind your back, please. | We just did it behind your back. | Follow five metres behind me. | ", 'pos': 'PREP'} 
 en retard sur {'examples': 'behind | We’re behind them in technology. | ', 'pos': 'PREP'}
за en retard sur examples_sim 0.7584283023672767 

INNER_MERGER_DICT
{'позади': {'avec': 0.5599654602031934, 'juste derrière': 0.8477911302459502, 'en retard': 0.6358701026292026, 'en retard sur': 0.848961





 99%|█████████▉| 181/182 [10:54<00:02,  2.09s/it]



100%|██████████| 182/182 [10:54<00:00,  3.60s/it]

под inférieur examples_sim 0.4075649930977122 

INNER_MERGER_DICT
{'под': {'inférieur': 0.4075649930977122}}

{'inférieur': {'под': 0.4075649930977122}}

FIRST LANG TO SECOND
под sorted_sim[0][1] 0.4075649930977122

 {'inférieur': ['под']}

{'под': ['inférieur']}

SECOND LANG TO FIRST
inférieur sorted_sim[0][1] 0.4075649930977122

 {'inférieur': ['под']}

{'под': ['inférieur']}

inférieur
all_clear inférieur
all_clear под
под
all_clear под
all_clear inférieur
pricessinf word best
INNER_MERGER_DICT
{}

{}

FIRST LANG TO SECOND

 {}

{}

SECOND LANG TO FIRST

 {}

{}

2162 2162 2162


In [78]:
a = list(df_all_words.iloc[1])

In [81]:
a[3]

'нужно'

In [92]:
df_all_words_azure.head(30)

,eng_word,l1,l1_context,l2,l2_context
0,necessary,mal nécessaire,necessary |,необходимые,necessary | required | needed | essential | ap...
1,necessary,mal nécessaire,necessary |,нужно,need | should | want | needs | must | needed |...
2,necessary,mal nécessaire,necessary |,требуется,required | needed | need | takes | necessary |...
3,necessary,mal nécessaire,necessary |,надо,should | necessary | Nothing is necessary to m...
4,necessary,si nécessaire,"if necessary | necessary | If necessary, we’ll...",обязательно,necessarily | will definitely | necessary | re...
5,necessary,si nécessaire,"if necessary | necessary | If necessary, we’ll...",необходимые,necessary | required | needed | essential | ap...
6,necessary,nécessaire,necessary | writing case | The Army needs men ...,необходимые,necessary | required | needed | essential | ap...
7,necessary,inévitable,necessary | inevitable | unavoidable | inescap...,необходимые,necessary | required | needed | essential | ap...
8,necessary,mal nécessaire,necessary |,необходимые,necessary | required | needed | essential | ap...
9,necessary,nécessaire,necessary | writing case | The Army needs men ...,необходимые,necessary | required | needed | essential | ap...


In [93]:
df_all_words_azure.to_csv("azure_cross_approach.csv")